In [ ]:
import tarfile
import os

# Define the file path and the extraction path
file_path = 'ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz'
extract_path = 'model'

# Extract the tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

# List the extracted files
extracted_files = os.listdir(extract_path)
print(extracted_files)


In [ ]:
extracted_dir_path = os.path.join(extract_path, 'ssdlite_mobiledet_cpu_320x320_coco_2020_05_19')
extracted_dir_contents = os.listdir(extracted_dir_path)
print(extracted_dir_contents)


In [5]:
import tensorflow as tf
import os

def generate_labels_from_tflite(model_path, output_labels_path):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get details of the output tensors
    output_details = interpreter.get_output_details()

    # Extract number of output classes
    output_tensor = interpreter.tensor(output_details[0]['index'])()
    num_classes = output_tensor.shape[-1]

    # Generate basic labels
    labels = [f"Label_{i}" for i in range(num_classes)]

    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_labels_path), exist_ok=True)

    # Save labels to a file
    with open(output_labels_path, 'w') as f:
        for label in labels:
            f.write(f"{label}\n")

    print(f"Labels saved to {output_labels_path}")

# Example usage with os.path for proper path handling
current_dir = os.path.dirname(os.path.abspath(__file__))
model_path = os.path.join(current_dir, 'model', 'ssdlite_mobiledet_cpu_320x320_coco_2020_05_19', 'model.tflite')
output_labels_path = os.path.join(current_dir, 'txt_output', 'labels.txt')

generate_labels_from_tflite(model_path, output_labels_path)


NameError: name '__file__' is not defined

In [ ]:
import tensorflow as tf
import os

# # Extracted directory path (adjust as needed)
# extracted_dir_path = os.path.join('model/ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18')

# Load the frozen inference graph
model_path = os.path.join(extracted_dir_path, 'tflite_graph.pb')

# Load the model
def load_frozen_model(model_file):
    with tf.io.gfile.GFile(model_file, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

# Load the frozen model
graph_def = load_frozen_model(model_path)

# Function to classify layers
def classify_layers(graph_def):
    input_layers = []
    hidden_layers = []
    output_layers = []

    for node in graph_def.node:
        # Identify input layer
        if 'input' in node.name.lower() or node.op in ['Placeholder']:
            input_layers.append(f"Layer Name: {node.name}, Layer Type: {node.op}")
        # Identify output layer
        elif 'output' in node.name.lower() or node.op in ['Softmax', 'Sigmoid', 'Identity', 'ArgMax', 'BBox', 'Decode']:
            output_layers.append(f"Layer Name: {node.name}, Layer Type: {node.op}")
        # Other layers are considered hidden layers
        else:
            hidden_layers.append(f"Layer Name: {node.name}, Layer Type: {node.op}")

    # Further refine input layers (typically only 1 or a few)
    meaningful_input_layers = [layer for layer in input_layers if 'input' in layer.lower()]

    # Further refine output layers (filter out intermediate outputs)
    meaningful_output_layers = [layer for layer in output_layers if 'output' in layer.lower() or 'classes' in layer.lower() or 'bbox' in layer.lower() or 'raw' in layer.lower()]

    return meaningful_input_layers, hidden_layers, meaningful_output_layers

# Classify the layers
input_layers, hidden_layers, output_layers = classify_layers(graph_def)

# Print number of each type of layer
print(f"Total number of input layers: {len(input_layers)}")
print(f"Total number of hidden layers: {len(hidden_layers)}")
print(f"Total number of output layers: {len(output_layers)}")

# Print input layers
print("\nInput Layers:")
for layer in input_layers:
    print(layer)

# Print hidden layers
print("\nHidden Layers:")
for layer in hidden_layers:
    print(layer)

# Print output layers
print("\nOutput Layers:")
for layer in output_layers:
    print(layer)
